In [1]:
# ! pip3 install fastai==2.7.9
# ! pip3 install torch==1.12.0+cu116 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.9.0+cu111
True


In [3]:
!nvidia-smi

Tue Mar 14 12:43:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P8    13W /  N/A |     33MiB /  6144MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
ROOT_PATH = '/home/yasaisen/Desktop/lab_02/step_focused_rgb'

In [5]:
import torch
import numpy as np
import os
from fastai.vision import *
from fastai.metrics import *
from fastai.vision.all import *
from fastai.data.external import untar_data,URLs
from fastai.data.transforms import get_image_files
import fastai
from fastai import callback
from fastai.data.core import DataLoaders

In [6]:
def checkpath(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
# model

Version = 'M_step4.2.0_f_rgb_3'

root_folder = os.path.abspath(os.path.join(ROOT_PATH, Version))

model_DIR = os.path.abspath(os.path.join(root_folder, 'model'))
checkpath(model_DIR)

In [8]:
# classes = ['NORMAL', 'RLN', 'RSLN']
# def get_label(GET, GETT):
#   if GET == 'NORMAL': 
#     return 0
#   elif GET == 'RLN' : 
#     return 1
#   elif GET == 'RSLN': 
#     return 2

In [9]:
classes = ['NORMAL', 'R', 'L']
def get_label(GET, GETT):
  if GET == 'NORMAL': 
    return 0
  
  if GETT == 'R' : 
    return 1
  elif GETT == 'L': 
    return 2

In [10]:
# train_data = 'step_train'
# valid_data = 'valid'
# test_data = 'test'
# koko_dir = ROOT_PATH

# train_data = 'step_train_rgb'
# valid_data = valid_rgb'
# test_data = 'test_rgb'
# koko_dir = ROOT_PATH

# train_data = 'step_train_focused'
# valid_data = 'valid_focused'
# test_data = 'test_focused'
# koko_dir = ROOT_PATH

train_data = 'step_train_rgb_focused'
valid_data = 'valid_rgb_focused'
koko_dir = ROOT_PATH

In [11]:
dblock = DataBlock( blocks = (ImageBlock, CategoryBlock),
                    get_items=partial(get_image_files,folders=[train_data, valid_data]),
                    splitter=FuncSplitter(lambda s: Path(s).parent.name == valid_data),
                    get_y=lambda x:get_label(x.stem.split('_')[0], x.stem.split('_')[1]),
                    item_tfms=Resize(540),
                    batch_tfms=[*aug_transforms(size=540, max_warp=0), Normalize.from_stats(*imagenet_stats)])

In [12]:
dls = dblock.dataloaders(source=koko_dir, bs=24)

In [13]:
metrics = [accuracy]

callback_fns = [callback.tracker.ShowGraphCallback]

learner = vision_learner(dls, models.resnet34, metrics=metrics, loss_func=CrossEntropyLossFlat())#, callback_fns=callback_fns)
# learner.model_dir = model_DIR

/home/yasaisen/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [14]:
learner.lr_find()

RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 5.81 GiB total capacity; 4.25 GiB already allocated; 4.44 MiB free; 4.32 GiB reserved in total by PyTorch)

In [ ]:
learner.fit_one_cycle(8, 0.0020892962347716093)

In [ ]:
learner.save(model_DIR)

# Predictions

In [ ]:
learner.show_results(figsize=(15,15))

In [ ]:
learner.recorder.plot_loss()

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix()

In [ ]:
dblock_valid = DataBlock( blocks = (ImageBlock, CategoryBlock),
                    get_items = get_files,
                    get_y = lambda x:get_label(x.stem.split('_')[0], x.stem.split('_')[1]),
                    item_tfms = Resize(540))

In [ ]:
dls_valid = dblock_valid.dataloaders(source=os.path.join(ROOT_PATH, valid_data), bs=24)

In [ ]:
interp = ClassificationInterpretation.from_learner(learner, dls_valid)
interp.plot_confusion_matrix()

In [ ]:
dblock_test = DataBlock( blocks = (ImageBlock, CategoryBlock),
                    get_items = get_files,
                    get_y = lambda x:get_label(x.stem.split('_')[0], x.stem.split('_')[1]),
                    item_tfms = Resize(540))

In [ ]:
dls_test = dblock_test.dataloaders(source=os.path.join(ROOT_PATH, test_data), bs=24)

In [ ]:
interp = ClassificationInterpretation.from_learner(learner, dls_test)
interp.plot_confusion_matrix()